In [1]:
# Nativos
import os
import sys

#calculo
import numpy as np
import pandas as pd

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

In [2]:
!ls data

sample_submission.csv  test_clean.scv  test.csv  train_clean.scv  train.csv


In [3]:
train = pd.read_csv('data/train_clean.scv', **set_parameter_csv)
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,...,Property_Area,Loan_Status,term_mount_ratio,ratio_mount,mount_fusion,gender_self_employed_fusion,gender_married_fusion,property_area_self_employed_fusion,education_gender_fusion,gender_married_dependents_fusion
0,LP001002,0.693252,0.629108,0.689855,0.708333,0.686000,5849,0.0,0.0,360.0,...,0.658416,1,0.696335,0.500000,0.500000,0.69403,0.623077,0.668675,0.720745,0.633028
1,LP001003,0.693252,0.718204,0.647059,0.708333,0.686000,4583,1508.0,128.0,360.0,...,0.614525,0,0.696335,0.685065,0.693878,0.69403,0.718663,0.601399,0.720745,0.652778
2,LP001005,0.693252,0.718204,0.689855,0.708333,0.682927,3000,0.0,66.0,360.0,...,0.658416,1,0.696335,0.703008,0.716667,0.68254,0.718663,0.583333,0.720745,0.744966
3,LP001006,0.693252,0.718204,0.689855,0.611940,0.686000,2583,2358.0,120.0,360.0,...,0.658416,1,0.696335,0.703008,0.716667,0.69403,0.718663,0.668675,0.601770,0.744966
4,LP001008,0.693252,0.629108,0.689855,0.708333,0.686000,6000,0.0,141.0,360.0,...,0.658416,1,0.696335,0.685065,0.693878,0.69403,0.623077,0.668675,0.720745,0.633028


In [4]:
test = pd.read_csv('data/test_clean.scv', **set_parameter_csv)
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,term_mount_ratio,ratio_mount,mount_fusion,gender_self_employed_fusion,gender_married_fusion,property_area_self_employed_fusion,education_gender_fusion,gender_married_dependents_fusion
0,LP001015,0.693252,0.718204,0.689855,0.708333,0.686,5720,0,110.0,360.0,0.795789,0.658416,0.696335,0.500000,0.500000,0.69403,0.718663,0.668675,0.720745,0.0
1,LP001022,0.693252,0.718204,0.647059,0.708333,0.686,3076,1500,126.0,360.0,0.795789,0.658416,0.696335,0.685065,0.693878,0.69403,0.718663,0.668675,0.720745,0.0
2,LP001031,0.693252,0.718204,0.752475,0.708333,0.686,5000,1800,208.0,360.0,0.795789,0.658416,0.696335,0.703008,0.716667,0.69403,0.718663,0.668675,0.720745,0.0
3,LP001035,0.693252,0.718204,0.752475,0.708333,0.686,2340,2546,100.0,360.0,0.740000,0.658416,0.696335,0.703008,0.716667,0.69403,0.718663,0.668675,0.720745,0.0
4,LP001051,0.693252,0.629108,0.689855,0.611940,0.686,3276,0,78.0,360.0,0.795789,0.658416,0.696335,0.685065,0.693878,0.69403,0.623077,0.668675,0.601770,0.0


In [5]:
X_train = train.drop(['Loan_ID', 'Loan_Status'], axis=1)
y_train = train['Loan_Status']
X_test = test.drop(['Loan_ID'], axis=1)

In [7]:
(X_train.columns == X_test.columns).any()

True

In [8]:
X_train.columns.shape

(19,)

In [11]:
%%time

import lightgbm as lgb
import time
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

stratt_fold = StratifiedKFold(
    n_splits=5, shuffle=False, random_state=hash(time.time())
)

param_grid = {
    'max_depth': [5, 10, 15,], 
    'n_estimators': [100, 250],
    'bagging_fraction': [0.5, 0.75],
    'learning_rate': [0.01, 0.1]
}

lgb_regressor = lgb.LGBMClassifier(
    save_binary=True,
    max_bin=255,
    random_state=hash(time.time())
)

gbm_1 = GridSearchCV(lgb_regressor, param_grid, cv=stratt_fold, n_jobs=4, scoring='accuracy')
gbm_1.fit(X_train, y_train)

best_params_1 = gbm_1.best_params_
model_1 = gbm_1.best_estimator_
score_1 = gbm_1.best_score_
score_1

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an 

CPU times: user 668 ms, sys: 93.4 ms, total: 762 ms
Wall time: 51.5 s


In [12]:
score_1

0.8094462540716613

In [13]:
best_params_1

{'bagging_fraction': 0.5,
 'learning_rate': 0.01,
 'max_depth': 10,
 'n_estimators': 100}

In [19]:
lgbm_predict = gbm_1.best_estimator_.predict(X_test)
type(lgbm_predict)

/home/villacorta/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


numpy.ndarray

In [20]:
lgbm_predict

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,

In [25]:
test['Loan_Status'] = lgbm_predict
prediction = test[['Loan_ID', 'Loan_Status']]
prediction.head()

,Loan_ID,Loan_Status
0,LP001015,1
1,LP001022,1
2,LP001031,1
3,LP001035,1
4,LP001051,1


In [26]:
prediction['Loan_Status'].value_counts(dropna=False)

1    308
0     59
Name: Loan_Status, dtype: int64

In [27]:
dicc_status = {
    1: 'Y',
    0: 'N'
}
prediction['Loan_Status'] = prediction['Loan_Status'].copy().map(dicc_status)
prediction['Loan_Status'].value_counts(dropna=False)

/home/villacorta/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Y    308
N     59
Name: Loan_Status, dtype: int64

In [28]:
prediction.to_csv('data/up_intent.csv', index=False)